In [12]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import numpy as np


#Cargar los datos de MNIST

(X_entrenamiento, Y_entrenamiento), (X_pruebas, Y_pruebas) = mnist.load_data()

In [13]:
X_entrenamiento
Y_entrenamiento
X_pruebas
Y_pruebas

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

 Las imágenes en el dataset MNIST son de 28x28 píxeles. Para usarlas con una CNN en TensorFlow, necesitamos agregar una dimensión extra para el canal de color. 
 
**X_entrenamiento.shape[0]** es el número de imágenes en el conjunto de entrenamiento.
Cada imagen se convierte a un tensor de forma **(28, 28, 1), donde 1 representa que es una imagen en escala de grises** (un solo canal de color).

**One-hot encoding**: Convertimos las etiquetas (que originalmente son números del 0 al 9) en vectores binarios donde solo el índice correspondiente a la etiqueta será 1, y el resto 0. 

**astype('float32')** convierte los datos de tipo entero (0-255) a flotante para permitir el uso de decimales.

In [14]:
X_entrenamiento = X_entrenamiento.reshape(X_entrenamiento.shape[0], 28, 28, 1)
X_pruebas = X_pruebas.reshape(X_pruebas.shape[0], 28, 28, 1)

Y_entrenamiento = to_categorical(Y_entrenamiento)
Y_pruebas = to_categorical(Y_pruebas)

X_entrenamiento = X_entrenamiento.astype('float32') / 255
X_pruebas = X_pruebas.astype('float32') / 255

**rango_rotacion**: Las imágenes pueden ser rotadas hasta 30 grados en cualquier dirección.

**mov_ancho**: 0.25. Permite mover la imagen horizontalmente hasta un 25% de su ancho.

**mov_alto**: 0.25. Permite mover la imagen verticalmente hasta un 25% de su altura.

**rango_acercamiento**: [0.5, 1.5]. Permite hacer zoom en la imagen, desde un 50% de su tamaño original hasta un 150%.

Usamos la clase **ImageDataGenerator** para variabilidad en las imágenes de entrenamiento.

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

(rango_rotacion,mov_ancho, mov_alto, rango_acercamiento) =  (30, 0.25,0.25, [0.5,1.5])

datagen = ImageDataGenerator(
    rotation_range = rango_rotacion,
    width_shift_range = mov_ancho,
    height_shift_range = mov_alto,
    zoom_range=rango_acercamiento,
)

datagen.fit(X_entrenamiento)

### **Modelo**

In [19]:
# Definición del modelo de CNN para clasificar dígitos de MNIST
model = tf.keras.models.Sequential([
    # Primera capa convolucional: 32 filtros de 3x3, ReLU, entrada de 28x28x1
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    # Reducción de dimensionalidad con pooling máximo
    tf.keras.layers.MaxPooling2D(2, 2),

    # Segunda capa convolucional: 64 filtros de 3x3, ReLU
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    # Otra reducción de dimensionalidad
    tf.keras.layers.MaxPooling2D(2,2),

    # Regularización para prevenir sobreajuste
    tf.keras.layers.Dropout(0.5),
    # Aplanar los datos 2D para la capa densa
    tf.keras.layers.Flatten(),
    # Capa densa intermedia con 100 neuronas y ReLU
    tf.keras.layers.Dense(100, activation='relu'),
    # Capa de salida: 10 neuronas (una por dígito), softmax para clasificación
    tf.keras.layers.Dense(10, activation="softmax")
])

# Compilación del modelo
model.compile(
    # Optimizador Adam para ajustar los pesos
    optimizer='adam',
    # Función de pérdida para clasificación multiclase con etiquetas one-hot
    loss='categorical_crossentropy',
    # Métrica de precisión para evaluar el rendimiento
    metrics=['accuracy']
)

In [20]:
data_gen_entrenamiento = datagen.flow(X_entrenamiento, Y_entrenamiento, batch_size=32)

In [22]:
TAMANO_LOTE = 32

print("Entrenando modelo...");
epocas=60
history = model.fit(
    data_gen_entrenamiento,
    epochs=epocas,
    batch_size=TAMANO_LOTE,
    validation_data=(X_pruebas, Y_pruebas),
    steps_per_epoch=int(np.ceil(60000 / float(TAMANO_LOTE))),
    validation_steps=int(np.ceil(10000 / float(TAMANO_LOTE)))
)

print("Modelo entrenado!");

Entrenando modelo...
Epoch 1/60
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 96s 50ms/step - accuracy: 0.4481 - loss: 1.5803 - val_accuracy: 0.9561 - val_loss: 0.1724
Epoch 2/60
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 130s 69ms/step - accuracy: 0.7583 - loss: 0.7469 - val_accuracy: 0.9599 - val_loss: 0.1405
Epoch 3/60
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 125s 66ms/step - accuracy: 0.8014 - loss: 0.6112 - val_accuracy: 0.9722 - val_loss: 0.0928
Epoch 4/60
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 155s 82ms/step - accuracy: 0.8237 - loss: 0.5451 - val_accuracy: 0.9774 - val_loss: 0.0820
Epoch 5/60
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 125s 67ms/step - accuracy: 0.8382 - loss: 0.5012 - val_accuracy: 0.9742 - val_loss: 0.0888
Epoch 6/60
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 128s 68ms/step - accuracy: 0.8445 - loss: 0.4769 - val_accuracy: 0.9691 - val_loss: 0.1003
Epoch 7/60
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 127s 68ms/step - accuracy: 0.8529 - loss: 0.4527 - val_accuracy: 0.9749 - val_loss: 0.0773
Epoch 8/60
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 128s 68

In [23]:
model.save('models/numeros_conv_ad_do.h5')